# Variables

## Import des librairies

In [ ]:
import os
import sys
from rdflib import Namespace, URIRef

## Définition des variables globales

In [ ]:
# Fichiers existants
ont_file_name = "ontology.ttl" # Fusion des deux fichiers précédents
ruleset_file_name = "rules.pie"

# Fichiers créés durant le processus
export_file_name = "addresses-temp.ttl"
out_file_name = "addresses.ttl"
local_config_file_name = "config_repo.ttl"
facts_ttl_file_name = "facts_data.ttl"
implicit_to_facts_ttl_file_name = "implicit_to_facts.ttl"

# Dossiers existants
data_folder_name = "data"
mapping_folder_name = "mappings"

# Dossiers créés durant le traitement
tmp_folder_name = "tmp_files"

# Nom du répertoire GraphDB pour les faits
facts_repository_name = "addresses_from_factoids"

# Définition des graphes nommés
ontology_named_graph_name = "ontology"
facts_named_graph_name = "facts"
factoids_named_graph_name = "factoids"
time_named_graph_name = "time"
permanent_named_graph_name = "permanent"
tmp_named_graph_name = "temporary"

# Définition de namespaces et de préfixes associés
namespace_prefixes = {"addr":Namespace("http://rdf.geohistoricaldata.org/def/address#"),
                      "atype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/attributeType/"),
                      "ltype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/"),
                      "lrtype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/"),
                      "ctype":Namespace("http://rdf.geohistoricaldata.org/id/codes/address/changeType/"),
                      "facts":Namespace("http://rdf.geohistoricaldata.org/id/address/facts/"),
                      "factoids":Namespace("http://rdf.geohistoricaldata.org/id/address/factoids/"),
                      "time": Namespace("http://www.w3.org/2006/time#"),
                      "owl": Namespace("http://www.w3.org/2002/07/owl#"),
                      "prov": Namespace("http://www.w3.org/ns/prov#"),
                      "rico": Namespace("https://www.ica.org/standards/RiC/ontology#"),
                      "geo": Namespace("http://www.opengis.net/ont/geosparql#"),
                      "skos": Namespace("http://www.w3.org/2004/02/skos/core#"),
                      "rdf": Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
                      "rdfs": Namespace("http://www.w3.org/2000/01/rdf-schema#"),
                      "xsd": Namespace("http://www.w3.org/2001/XMLSchema#"),
                      "geofla": Namespace("http://data.ign.fr/def/geofla#"),
                      }

# URIs pour accéder ai logiciel GraphDB
graphdb_url = "http://localhost:7200"

py_code_folder_path = "../code"

## Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
mapping_folder = os.path.abspath(mapping_folder_name)
data_folder = os.path.abspath(data_folder_name)

python_code_folder = os.path.abspath(py_code_folder_path)

local_config_file = os.path.join(tmp_folder, local_config_file_name)
ont_file = os.path.abspath(ont_file_name)
ruleset_file = os.path.abspath(ruleset_file_name)
facts_ttl_file = os.path.join(tmp_folder, facts_ttl_file_name)
implicit_to_facts_ttl_file = os.path.join(tmp_folder, implicit_to_facts_ttl_file_name)

## Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import filemanagement as fm
import graphdb as gd
import attributeversioncomparisons as avc
import multisourcesprocessing as msp
import factoidscreation as fc
import timeprocessing as tp

## Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(tmp_folder)

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait.

In [ ]:
# Il se peut que la suppression d'un répertoire ne fonctionne pas donc pour éviter la suppresion au moment de la réinitialisation (suppression + (re)création)
# `allow_removal` doit valoir False et dans ce cas-là, le répertoire sera juste vidé.
allow_removal = False
disable_same_as = False

# gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_name="owl2-rl-optimized", disable_same_as=disable_same_as, allow_removal=allow_removal)
gd.reinitialize_repository(graphdb_url, facts_repository_name, local_config_file, ruleset_file=ruleset_file, disable_same_as=disable_same_as, allow_removal=allow_removal)

## Gestion du répertoire local

## Import des ontologies

In [ ]:
gd.load_ontologies(graphdb_url, facts_repository_name, [ont_file], ontology_named_graph_name)

## Définition de variables liées aux sources

### Voies de Paris via Wikidata

* `wd` pour "wikidata"
* `wdp_land` pour "wikidata paris landmarks"
* `wdp_loc` pour "wikidata paris locations"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de Wikidata
wd_repository_name = "factoids_wikidata"

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdp_land_csv_file_name = "wd_paris_landmarks.csv"
wdp_land_csv_file = os.path.join(data_folder, wdp_land_csv_file_name)

# Fichier CSV pour stocker le résultat de la requête de la sélection
wdp_loc_csv_file_name = "wd_paris_locations.csv"
wdp_loc_csv_file = os.path.join(data_folder, wdp_loc_csv_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
wdp_kg_file_name = "wd_paris.ttl"
wdp_kg_file = os.path.join(tmp_folder, wdp_kg_file_name)

# Fichier TTL final des factoïdes de Wikidata
wdp_factoids_kg_file_name = "wd_paris_factoids.ttl"
wdp_factoids_kg_file = os.path.join(tmp_folder, wdp_factoids_kg_file_name)
wdp_permanent_kg_file_name = "wd_paris_permanent.ttl"
wdp_permanent_kg_file = os.path.join(tmp_folder, wdp_permanent_kg_file_name)

### Nomenclature des voies de la ville de Paris

Les données de la ville de Paris sont composées de deux jeux :
* [dénominations des emprises des voies actuelles](https://opendata.paris.fr/explore/dataset/denominations-emprises-voies-actuelles)
* [dénominations caduques des voies](https://opendata.paris.fr/explore/dataset/denominations-des-voies-caduques)

Les voies actuelles ont une emprise géométrique contrairement aux anciennes voies.

* `vpt` pour "ville paris thoroughfares"
* `vpta` pour "ville paris thoroughfares actuelles"
* `vptc` pour "ville paris thoroughfares caduques"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
vpt_repository_name = "factoids_ville_de_paris"

# Fichier CSV pour stocker le résultat de la requête de la sélection
vpta_csv_file_name = "denominations-emprises-voies-actuelles.csv"
vpta_csv_file = os.path.join(data_folder, vpta_csv_file_name)
vptc_csv_file_name = "denominations-des-voies-caduques.csv"
vptc_csv_file = os.path.join(data_folder, vptc_csv_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
vpt_kg_file_name = "voies_paris.ttl"
vpt_kg_file = os.path.join(tmp_folder, vpt_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
vpt_factoids_kg_file_name = "vpt_factoids.ttl"
vpt_factoids_kg_file = os.path.join(tmp_folder, vpt_factoids_kg_file_name)
vpt_permanent_kg_file_name = "vpt_permanent.ttl"
vpt_permanent_kg_file = os.path.join(tmp_folder, vpt_permanent_kg_file_name)

vpt_time_description = {
    "start_time" : {"stamp":"2024-02-10T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### Base Adresse Nationale (BAN)

Données de la [Base Adresse Nationale (BAN)](https://adresse.data.gouv.fr/base-adresse-nationale), accessible [ici](https://adresse.data.gouv.fr/data/ban/adresses/latest/csv)

`bpa` pour "BAN paris addresses"

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données de la BAN
bpa_repository_name = "factoids_ban"

# Fichier CSV pour stocker le résultat de la requête de la sélection
bpa_csv_file_name = "ban_adresses.csv"
bpa_csv_file = os.path.join(data_folder, bpa_csv_file_name)

# Fichier TTL pour structurer les connaissances des voies de Paris
bpa_kg_file_name = "ban_adresses.ttl"
bpa_kg_file = os.path.join(tmp_folder, bpa_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
bpa_factoids_kg_file_name = "ban_factoids.ttl"
bpa_factoids_kg_file = os.path.join(tmp_folder, bpa_factoids_kg_file_name)
bpa_permanent_kg_file_name = "ban_permanent.ttl"
bpa_permanent_kg_file = os.path.join(tmp_folder, bpa_permanent_kg_file_name)

bpa_time_description = {
    "start_time" : {"stamp":"2024-01-01T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### OpenStreetMap (OSM)

Extraction des données d'OpenStreetMap

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données d'OSM
osm_repository_name = "factoids_osm"

# Fichiers CSV pour stocker les résultats des requêtes de la sélection
osm_csv_file_name = "osm_adresses.csv"
osm_csv_file = os.path.join(data_folder, osm_csv_file_name)
osm_hn_csv_file_name = "osm_hn_adresses.csv"
osm_hn_csv_file = os.path.join(data_folder, osm_hn_csv_file_name)

# Fichiers TTL pour structurer les connaissances venant d'OSM
osm_kg_file_name = "osm_adresses.ttl"
osm_kg_file = os.path.join(tmp_folder, osm_kg_file_name)

# Fichier TTL final des factoïdes de Ville de Paris
osm_factoids_kg_file_name = "osm_factoids.ttl"
osm_factoids_kg_file = os.path.join(tmp_folder, osm_factoids_kg_file_name)
osm_permanent_kg_file_name = "osm_permanent.ttl"
osm_permanent_kg_file = os.path.join(tmp_folder, osm_permanent_kg_file_name)

osm_time_description = {
    "start_time" : {"stamp":"2024-01-01T00:00:00Z","precision":"day","calendar":"gregorian"}
    }

### Intégration de données issues de fichiers Geojson

Ces fichiers sont issus de la vectorisation de plans de Paris
* cadatre napoléonien révisé de 1847
* plan "Andriveau" de 1849
* plan parcellaire municipal de 1871
* plan de l'Atlas Municipal de 1888

#### Variables globales pour l'import des données venant des fichiers Geojson

In [ ]:
lang = "fr"
landmark_type = "Thoroughfare"
geojson_join_property = "name"

#### Cadastre napoléonien de 1847

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
cn_1847_repository_name = "factoids_1847_cadastre_nap"

cn_1847_geojson_file_name = "1847_cadastre_nap.geojson"
cn_1847_geojson_file = os.path.join(data_folder, cn_1847_geojson_file_name)
cn_1847_kg_file_name = "cn_1847_kg.ttl"
cn_1847_kg_file = os.path.join(tmp_folder, cn_1847_kg_file_name)

# Fichier TTL final des factoïdes du cadatre napoléonien révisé de 1847
cn_1847_factoids_kg_file_name = "cn_1847_factoids.ttl"
cn_1847_factoids_kg_file = os.path.join(tmp_folder, cn_1847_factoids_kg_file_name)
cn_1847_permanent_kg_file_name = "cn_1847_permanent.ttl"
cn_1847_permanent_kg_file = os.path.join(tmp_folder, cn_1847_permanent_kg_file_name)

cn_1847_geojson = fm.read_json_file(cn_1847_geojson_file)

cn_1847_source_desc = {
    "lang" : "fr", 
    "label" : "Cadastre napoléonien de Gentilly de 1847",
    "publisher" : {
        "label": "Empire français"
        }
}

cn_1847_time_interval = {
    "start_time" : {"stamp":"1845-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1850-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
cn_1847_geojson["source"] = cn_1847_source_desc
cn_1847_geojson["time"] = cn_1847_time_interval


#### Plan d'Andriveau

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
an_1849_repository_name = "factoids_1849_andriveau"

an_1849_geojson_file_name = "1849_andriveau.geojson"
an_1849_geojson_file = os.path.join(data_folder, an_1849_geojson_file_name)
an_1849_kg_file_name = "an_1849_kg.ttl"
an_1849_kg_file = os.path.join(tmp_folder, an_1849_kg_file_name)

# Fichier TTL final des factoïdes du plan "Andriveau" de 1849
an_1849_factoids_kg_file_name = "an_1849_factoids.ttl"
an_1849_factoids_kg_file = os.path.join(tmp_folder, an_1849_factoids_kg_file_name)
an_1849_permanent_kg_file_name = "an_1849_permanent.ttl"
an_1849_permanent_kg_file = os.path.join(tmp_folder, an_1849_permanent_kg_file_name)

an_1849_geojson = fm.read_json_file(an_1849_geojson_file)

an_1849_source_desc = {
    "lang" : "fr", 
    "label" : "Plan d'Andriveau de 1849",
    "publisher" : {
        "label": "Andriveau"
        }
}

an_1849_time_interval = {
    "start_time" : {"stamp":"1847-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1851-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
an_1849_geojson["source"] = an_1849_source_desc
an_1849_geojson["time"] = an_1849_time_interval

#### Plan parcellaire municipal de 1871

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
pm_1871_repository_name = "factoids_1871_plan_parcellaire_mun"

pm_1871_geojson_file_name = "1871_plan_parcellaire_mun.geojson"
pm_1871_geojson_file = os.path.join(data_folder, pm_1871_geojson_file_name)
pm_1871_kg_file_name = "pm_1871_kg.ttl"
pm_1871_kg_file = os.path.join(tmp_folder, pm_1871_kg_file_name)

# Fichier TTL final des factoïdes du plan parcellaire municipal de 1871
pm_1871_factoids_kg_file_name = "pm_1871_factoids.ttl"
pm_1871_factoids_kg_file = os.path.join(tmp_folder, pm_1871_factoids_kg_file_name)
pm_1871_permanent_kg_file_name = "pm_1871_permanent.ttl"
pm_1871_permanent_kg_file = os.path.join(tmp_folder, pm_1871_permanent_kg_file_name)

pm_1871_geojson = fm.read_json_file(pm_1871_geojson_file)

pm_1871_source_desc = {
    "lang" : "fr",
    "label" : "Plan parcellaire municipal",
    "publisher" : {
        "label": "IIIe République"
        }
}

pm_1871_time_interval = {
    "start_time" : {"stamp":"1870-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1872-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
pm_1871_geojson["source"] = pm_1871_source_desc
pm_1871_geojson["time"] = pm_1871_time_interval

#### Plan municipal de 1888

In [ ]:
# Nom du répertoire où sont stockés et construits les triplets des factoïdes des données
am_1888_repository_name = "factoids_1888_atlas_municipal"

am_1888_geojson_file_name = "1888_atlas_municipal.geojson"
am_1888_geojson_file = os.path.join(data_folder, am_1888_geojson_file_name)
am_1888_kg_file_name = "am_1888_kg.ttl"
am_1888_kg_file = os.path.join(tmp_folder, am_1888_kg_file_name)

# Fichier TTL final des factoïdes du plan de l'Atlas Municipal de 1888
am_1888_factoids_kg_file_name = "am_1888_factoids.ttl"
am_1888_factoids_kg_file = os.path.join(tmp_folder, am_1888_factoids_kg_file_name)
am_1888_permanent_kg_file_name = "am_1888_permanent.ttl"
am_1888_permanent_kg_file = os.path.join(tmp_folder, am_1888_permanent_kg_file_name)

am_1888_geojson = fm.read_json_file(am_1888_geojson_file)

am_1888_source_desc = {
    "lang" : "fr", 
    "label" : "Plan de l'atlas municipal de 1888",
    "publisher" : {
        "label": "Ville de Paris"
        }
}

am_1888_time_interval = {
    "start_time" : {"stamp":"1887-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
    "end_time" : {"stamp":"1889-01-01T00:00:00Z","precision":"year","calendar":"gregorian"},
}
am_1888_geojson["source"] = am_1888_source_desc
am_1888_geojson["time"] = am_1888_time_interval

## Processus final et itératif

### Création des factoïdes dans des répertoires

Pour chaque source, des factoïdes sont créés indépendamment dans des répertoires distincts

#### Ville de Paris


In [ ]:
fc.create_factoids_repository_ville_paris(graphdb_url, vpt_repository_name, namespace_prefixes, tmp_folder,
                                          ont_file, ontology_named_graph_name,
                                          factoids_named_graph_name, permanent_named_graph_name,
                                          vpta_csv_file, vptc_csv_file, vpt_kg_file, vpt_time_description, lang=lang)

####  BAN


In [ ]:
fc.create_factoids_repository_ban(graphdb_url, bpa_repository_name, namespace_prefixes, tmp_folder,
                                  ont_file, ontology_named_graph_name,
                                  factoids_named_graph_name, permanent_named_graph_name,
                                  bpa_csv_file, bpa_kg_file, bpa_time_description, lang=lang)

#### Wikidata


In [ ]:
fc.create_factoids_repository_wikidata_paris(graphdb_url, wd_repository_name, namespace_prefixes, tmp_folder,
                                             ont_file, ontology_named_graph_name,
                                             factoids_named_graph_name, permanent_named_graph_name,
                                             wdp_land_csv_file, wdp_loc_csv_file, wdp_kg_file, wdp_time_description={}, lang=lang)

#### OSM

In [ ]:
fc.create_factoids_repository_osm(graphdb_url, osm_repository_name, namespace_prefixes, tmp_folder,
                                  ont_file, ontology_named_graph_name,
                                  factoids_named_graph_name, permanent_named_graph_name,
                                  osm_csv_file, osm_hn_csv_file, osm_kg_file, osm_time_description=osm_time_description, lang=None)

#### Données issues des fichiers Geojson

* cadastre napoléonien de Gentilly (1847)
* plan d'Andriveau (1849)
* plan parcellaire municipal de Paris (1871)
* plan municipal de Paris (1888)

In [ ]:
fc.create_factoids_repository_geojson_states(graphdb_url, cn_1847_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, cn_1847_geojson, geojson_join_property, cn_1847_kg_file, landmark_type, lang)
fc.create_factoids_repository_geojson_states(graphdb_url, an_1849_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, an_1849_geojson, geojson_join_property, an_1849_kg_file, landmark_type, lang)
fc.create_factoids_repository_geojson_states(graphdb_url, pm_1871_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, pm_1871_geojson, geojson_join_property, pm_1871_kg_file, landmark_type, lang)
fc.create_factoids_repository_geojson_states(graphdb_url, am_1888_repository_name, namespace_prefixes, tmp_folder, ont_file, ontology_named_graph_name,
                               factoids_named_graph_name, permanent_named_graph_name, am_1888_geojson, geojson_join_property, am_1888_kg_file, landmark_type, lang)

### Insertion des factoïdes dans le graphe des faits

#### Ville de Paris

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, vpt_repository_name, vpt_factoids_kg_file, vpt_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

#### Wikidata

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, wd_repository_name, wdp_factoids_kg_file, wdp_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

#### BAN

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, bpa_repository_name, bpa_factoids_kg_file, bpa_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

#### OSM

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, osm_repository_name, osm_factoids_kg_file, osm_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

#### Données issues des fichiers Geojson

* cadastre napoléonien de Gentilly (1847)
* plan d'Andriveau (1849)
* plan parcellaire municipal de Paris (1871)
* plan municipal de Paris (1888)

In [ ]:
msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, cn_1847_repository_name,
                                       cn_1847_factoids_kg_file, cn_1847_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, an_1849_repository_name,
                                       an_1849_factoids_kg_file, an_1849_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, pm_1871_repository_name,
                                       pm_1871_factoids_kg_file, pm_1871_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

msp.transfert_factoids_to_facts_repository(graphdb_url, facts_repository_name, am_1888_repository_name,
                                       am_1888_factoids_kg_file, am_1888_permanent_kg_file, factoids_named_graph_name, facts_named_graph_name, permanent_named_graph_name)
msp.import_factoids_in_facts(graphdb_url, facts_repository_name, namespace_prefixes, factoids_named_graph_name, facts_named_graph_name)

### Traitement des données temporelles

Ordonner les valeurs temporelles pour pouvoir "fusionner" (ajout d'un lien `owl:sameAs`) les versions d'attributs qui sont les mêmes. 

In [ ]:
# Ajout de relations temporelles
# tp.compare_events(graphdb_url, facts_repository_name, namespace_prefixes, time_named_graph_name)
tp.add_time_relations(graphdb_url, facts_repository_name, namespace_prefixes, time_named_graph_name)


time_named_graph_uri = URIRef(gd.get_named_graph_uri_from_name(graphdb_url, facts_repository_name, time_named_graph_name))
prefixes = gd.get_query_prefixes_from_namespaces(namespace_prefixes)
# time_instant_select_conditions = "?ev a addr:Event ; ?tpred1 ?ti1 ; ?tpred2 ?ti2 ."
# time_instant_select_conditions = """
#         ?attr a addr:Attribute ; addr:changedBy ?cg1, ?cg2.
#         ?cg1 a addr:AttributeChange ; addr:dependsOn [?tpred1 ?ti1] .
#         ?cg2 a addr:AttributeChange ; addr:dependsOn [?tpred2 ?ti2] .
#         """
# query = tp.get_query_to_compare_time_instants(time_named_graph_uri, prefixes, time_instant_select_conditions)

# time_interval_select_conditions = """
#         ?attr a addr:Attribute ; addr:hasAttributeVersion ?av1, ?av2 .
#         FILTER (?av1 != ?av2)
#         """
# query = tp.get_query_to_compare_time_intervals(time_named_graph_uri, prefixes, time_interval_select_conditions)
# print(query)

# tp.get_earliest_and_latest_time_instants_for_events(graphdb_url, facts_repository_name, prefixes, time_named_graph_uri)

Comparaison des versions d'attribut décrivant des géométries

In [ ]:
crs_uri = URIRef('http://www.opengis.net/def/crs/EPSG/0/2154')
comp_named_graph_name = "version_comparisons"
avc.compare_geometry_versions(graphdb_url, facts_repository_name, namespace_prefixes, facts_named_graph_name, comp_named_graph_name, crs_uri, buffer_radius=5)
avc.compare_name_versions(graphdb_url, facts_repository_name, namespace_prefixes, facts_named_graph_name, comp_named_graph_name, similarity_coef=0.85)

Fusion de versions

In [ ]:
query1 = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX time: <http://www.w3.org/2006/time#>
    INSERT {
        GRAPH ?g {
            ?s owl:sameAs ?o
        }
    }
    WHERE {
        BIND (<http://www.example.org/test> AS ?g)
        ?av1 a addr:AttributeVersion ; addr:hasTime ?i1 ; addr:isOutdatedBy ?cg1.
        ?av2 a addr:AttributeVersion ; addr:hasTime ?i2 ; addr:isMadeEffectiveBy ?cg2.
        ?av1 ?pred ?av2 .
        FILTER(?av1 != ?av2 && ?pred in (addr:sameVersionValueAs, addr:differentVersionValueFrom))
        ?i1 time:intervalBefore ?i2 .

        BIND(IF(?pred = addr:sameVersionValueAs, ?av1, ?cg1) AS ?s)
        BIND(IF(?pred = addr:sameVersionValueAs, ?av2, ?cg2) AS ?o)
        MINUS {
            ?av3 a addr:AttributeVersion ; addr:hasTime ?i3 .
            FILTER(?av1 != ?av3)
            FILTER(?av2 != ?av3)
            {
                ?i1 (time:intervalMeets|time:intervalOverlaps|time:intervalIn) ?i3 .
            } UNION {
                ?i1 (time:intervalMeets|time:intervalOverlaps|time:intervalIn) ?i2 .
            } UNION {
                ?i1 time:intervalBefore ?i3 .
                ?i2 time:intervalAfter ?i3 .
            }
        }
    }
"""

query2 = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX addr: <http://rdf.geohistoricaldata.org/def/address#>
    PREFIX time: <http://www.w3.org/2006/time#>
    INSERT {
        GRAPH ?g {
            ?cg1end owl:sameAs ?cg2beg .
        }
    } WHERE {
        BIND (<http://www.example.org/test> AS ?g)
        ?av1 a addr:AttributeVersion ; addr:hasTime ?i1 .
        ?av2 a addr:AttributeVersion ; addr:hasTime ?i2 .
        ?av1 addr:differentVersionValueFrom ?av2 .
        ?i1 ?p ?i2 .
        FILTER(?p in (time:intervalOverlaps, time:intervalMeets))
        ?cg1end addr:outdates ?av1 .
        ?cg2beg addr:makesEffective ?av2 .
    } ; 

    INSERT {
        GRAPH ?g {
            ?av1 owl:sameAs ?av2 .
        }
    } WHERE {
        BIND (<http://www.example.org/test> AS ?g)
        ?av1 a addr:AttributeVersion ; addr:hasTime ?i1 .
        ?av2 a addr:AttributeVersion ; addr:hasTime ?i2 .
        ?av1 addr:sameVersionValueAs ?av2 .
        ?i1 ?p ?i2 .
        FILTER(?p in (time:intervalOverlaps, time:intervalMeets, time:intervalIn, time:intervalEquals))
    }
    """

gd.update_query(query1, graphdb_url, facts_repository_name)
gd.update_query(query2, graphdb_url, facts_repository_name)